# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097008


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:40,  3.59s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<01:06,  2.57s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:46,  1.84s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:30,  1.33s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  5.86it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.21it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.86it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  3.92it/s]

ElisabethB                            0.040579
not-logged-in-3b0b929f4475fd276561    0.000919
Jnursssmith5263                       0.002247
jnarayanbvg                           0.005397
not-logged-in-94a5d4ef4c6d13e8839f    0.000319
awright5                              0.001527
srivishnu                             0.000787
not-logged-in-e5015a872e4a0f43792d    0.034377
rmz4160                               0.000640
bentleyghioda                         0.000245
not-logged-in-4447ff92852aa7c13340    0.000730
sn346808                              0.035282
not-logged-in-14d09f70263c1fe15cc3    0.138267
KrisKeogh                             0.001006
nsdeni                                0.000218
not-logged-in-a7658ef4de3eefd90ccb    0.007888
plankjames                            0.000163
umtuck1                               0.000236
HenriqueDornelles                     0.017903
MFinCatfish                           0.000169
BigSean                               0.000162
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())